In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import astropy.wcs
from astropy.io import fits
import astropy.io
from IPython.display import clear_output
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from matplotlib.backends.backend_pdf import PdfPages

df = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e8.csv")
#df = pd.read_csv("test8.csv")
filename_xml = "d_xml_gaia/sub" + df["ID"] + "_gaia.xml"
filename_fits = "d_fits/sub" + df["ID"] + ".fits"
output_filename = "sub" + df["ID"] + "_star.csv"
with open("d_xml_gaia/sub571226_095_gaia.xml") as f:
    a = f.readlines()
    b = a[116]#<TABLEDATA>
    c = a[-5] #</TABLEDATA>

###################################### xml ファイルの読み込み #######
#################################恒星の(ra, dec, ,source_ID, Gmag, Gmag_err)のリストにする###
#xml_all = np.arange(6).reshape(6)
#xml_all = np.arange(5).reshape(5)
for i in range(len(filename_xml)):
#for i in range(1):
    print filename_xml[i],i
    xml_all = np.arange(7).reshape(7)
    xml_list = np.arange(5).reshape(5)
    fits = astropy.io.fits.open(filename_fits[i])
    hdul = fits
    data = hdul[0].data
    data = data[1,:,:]#差分前のフレームだけにする
    data = np.reshape(data, (282, 500))#2次元化
    wcs = astropy.wcs.WCS(fits[0].header, naxis = 2)
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)
    daofind = DAOStarFinder(fwhm=3.0, threshold=30.*std)    ##############ここの設定をみる必要あり
    sources = daofind(data - median)
    for col in sources.colnames:
        sources[col].info.format = '%.8g'  # for consistent table output
    positions = (sources['xcentroid'], sources['ycentroid'])
    #apertures = CircularAperture(positions, r=4.)
    #plt.imshow(data, cmap = 'gray', vmin = 0, vmax = 400, interpolation = 'none')
    #apertures.plot(color='green', lw=1.5, alpha=0.5)
    #plt.ylim(0, 282)
    with open(filename_xml[i]) as f:
        l = f.readlines()
        for j in range(len(l)):
            m = l[j]
            if m == b:
                del_start = j + 1
            elif m == c:
                del_end = j
        l = l[del_start:del_end]
        xml = l[0]
        xml = re.split('<TR><TD>|</TD><TD>|</TD></TR>', xml)
        print xml
        del xml[0],xml[2:6],xml[3:12],xml[5:]
        print xml
        for k in l:
            s = re.split('<TR><TD>|</TD><TD>|</TD></TR>', k)
            del s[0],s[2:6],s[3:12],s[5:]
            if s != ['', '', '', '','']:
                xml_list = np.vstack((xml_list, s))
    
    xml_list = np.delete(xml_list,0, 0)
    #print xml_list
    source_ID = xml_list[:,2]
    source_ID = source_ID.reshape(len(source_ID),1)
    xml_list = xml_list.astype(np.float64)
    pixel_coordinates = np.arange(2).reshape(1,2)
    for k in range(len(xml_list)):
                pixel_coord = wcs.all_world2pix(xml_list[k,0], xml_list[k,1], 1)
                pixel_coordinates = np.vstack((pixel_coordinates, pixel_coord))
    pixel_coordinates = np.delete(pixel_coordinates,0, 0)
    
    #xml_list = np.hstack((xml_list,pixel_coordinates,source_ID))
    xml_list = np.hstack((xml_list,pixel_coordinates))
    xml_list = np.delete(xml_list,2,axis=1)
    
    #x座標がフレーム外のindex行をxmkから削除
    index = np.where((xml_list[:,4]>500.)|(xml_list[:,4]<0.)|\
                    (xml_list[:,5]>282.) | (xml_list[:,5]<0.))
    xml_list = np.delete(xml_list,index, 0)
    source_ID = np.delete(source_ID,index,0)
    #xml_list = np.hstack((xml_list,source_ID))
    #xml_all = np.vstack((xml_all,xml_list))
#####################################
# Source Extractor と比較
    pix_sextractor = np.array(positions)
    pix_sextractor = pix_sextractor.transpose()
    use_index = []
    for m in range(len(pix_sextractor)):
    #for m in range(1):
        index = abs(xml_list[:,4] - pix_sextractor[m,0])+\
                abs(xml_list[:,5] - pix_sextractor[m,1])
        a = index.argmin()
        use_index = np.hstack((use_index,a))
        use_index = use_index.astype(np.int64)
    
    xml_list = xml_list[use_index,:]
    source_ID = source_ID[use_index,:]
    xml_list = np.hstack((xml_list,source_ID))
    xml_all = np.vstack((xml_all,xml_list))
    #plt.imshow(data, cmap = 'gray', vmin = 0, vmax = 400, interpolation = 'none')
    #plt.scatter(xml_list[:,3],xml_list[:,4],alpha = 0.5)
    #apertures.plot(color='green', lw=1.5, alpha=0.5)
    #plt.ylim(0, 282)

### 今はsource_IDとxml_listをhstackさせちゃうけど　sextractor でどうせ
##まだできなくなるから,source_IDを別にしておく方も考えとく
#v = np.random.rand(26).reshape(13,2)*100
#print v.dtype
#print "aaa"
#print np.delete(v, np.where((v[:,0]>80.) | (v[:,0]<20.)), 0)
    xml_all = np.delete(xml_all,0,0)
    columns = ["Ra", "Dec", "GMag", "Gmag_err", "x", "y", "source_ID"]
    df = pd.DataFrame(data = xml_all, columns = columns)
    df.to_csv(output_filename[i])
clear_output()
print "Complete!"

Complete!
